# Sentinel Hub parameters

Uncomment the following line to install eotdl if needed.

In [ ]:
# !pip install eotdl

In order to search or download imagery through the Sentinel Hub API, we need to define some parameters, such as the Data Collection (`sentinel-1-grd`, `sentinel-2-l2a` ...), the EvalScript, the resolution and so on. In order to facilitate the definition of these parameters and abstract them from the sentinelhub library, the EOTDL has a class called `SHParametersFeature`, which is simply a Python dictionary with the variables that we want to define. It is almost identical to the `search` parameters that can be used in the sentinelhub Python library, but it is an abstraction from it.

In [1]:
from eotdl.access import SHParametersFeature

By default, we have wrapped those parameters in several Python class like `sentinel_2_l2a_download_parameters` or `sentinel_1_search_parameters`. With that, we just have to give the download dict with the bounding box and time interval and the folder where we want to download the data. And voilà! All it's managed by the EOTDL environment. For further information about the download or search dict, check this [notebook](11_sh_search.ipynb).


As said, we can import several wrapped parameters.

In [9]:
from eotdl.access.sentinelhub.parameters import sentinel_1_search_parameters, sentinel_2_l2a_download_parameters


And get the defined parameters.

In [10]:
print('Sentinel-1 search parameters:')
print(sentinel_1_search_parameters.data_collection)
print(sentinel_1_search_parameters.fields)
print('')
print('Sentinel-2 L2A download parameters:')
print(sentinel_2_l2a_download_parameters.data_collection)
print(sentinel_2_l2a_download_parameters.resolution)

Sentinel-1 search parameters:
DataCollection.SENTINEL1
{'include': ['id', 'properties.datetime', 'sar:instrument_mode', 's1:polarization', 'sat:orbit_state', 's1:resolution', 's1:timeliness'], 'exclude': []}

Sentinel-2 L2A download parameters:
DataCollection.SENTINEL2_L2A
(10, 10)


But the power behind this is that we can create our own `SHParametersFeature`, depending on our needs. The parameters that can be defined are the following:
- `data_collection`: sentinelhub.DataCollection object with the desired data collection to search or download.
- `data_to_download`: dict with the unique ID and bounding box of every location to request sentinel data from.
- `bbox`: list with the bounding box of the location.
- `time_interval`: list with the time interval stablished.
- `resolution`: resolution of the image to download.
- `data_folder`: location of the directory where the fetched data will be saved.
- `fields`: dict with the fields desired to include or exclude. It must follow the format:
    {
    "include": list of fields to include, 
    "exclude": list of fields to exclude
    }
- `filter`: string with the filter to apply, if needed.
- `evalscript`: is a piece of Javascript code which defines how the satellite data shall be 
processed by Sentinel Hub and what values the service shall return. For further information,
go [here](https://docs.sentinel-hub.com/api/latest/evalscript/). Some default evalscripts have been defined.
- `mosaicking_order`: sentinelhub.MosaickingOrder object with the desired mosaicking order.

Now let's image we want to create our own `SHParametersFeature` to search for `sentinel-2-l2a` imagery with a cloud filter. As easy as follows.

In [ ]:
from sentinelhub import DataCollection

options = {
        'data_collection': DataCollection.SENTINEL2_L2A,
        'fields': {"include": ["id", 
                               "properties.datetime",
                               "properties.eo:cloud_cover"],
            "exclude": [],
        'filter': "eo:cloud_cover < 5"
        }
    }
new_search_parameters = SHParametersFeature(options)

Or maybe we want to download `sentinel-2-l2a` imagery.

In [ ]:
from sentinelhub import MosaickingOrder
from eotdl.access.sentinelhub.evalscripts import EvalScripts   # Already defined EvalScripts

options = {
        'data_collection': DataCollection.SENTINEL2_L2A,
        'resolution': (10, 10),
        'mosaicking_order': MosaickingOrder.LEAST_CC,
        'evalscript': EvalScripts.SENTINEL_2,
        'fields': {"include": ["id", 
                               "properties.datetime",
                               "properties.eo:cloud_cover"],
            "exclude": [],
        'filter': "eo:cloud_cover < 5"
        }
    }
new_download_parameters = SHParametersFeature(options)

And what if we want to use our new parameters feature to search or download imagery? We first need to define our AoI and time interval. We can define them in the `SHParametersFeature` both givin a search or download dict or giving them directly.

In [12]:
dates = ('2020-01-01', '2020-01-10')
jaca_bounding_box = (-0.557034,42.569409,-0.547706,42.57571)

Give them as a dict.

In [ ]:
search_dict = {
    'Jaca': {
        'bounding_box': jaca_bounding_box,
        'time_interval': dates
    }
}

new_download_parameters.data_to_download = search_dict

Or give them directly.

In [ ]:
new_search_parameters.bounding_box = jaca_bounding_box
new_search_parameters.time_interval = dates